In [6]:
#!touch 7-densenet121.py | chmod +x *.py
#!pwd

In [2]:
import tensorflow.keras as K

In [3]:
#!/usr/bin/env python3

import tensorflow.keras as K
#inception_block = __import__('0-inception_block').inception_block
def inception_block(A_prev, filters):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    Conv2D = K.layers.Conv2D
    Concatenate = K.layers.Concatenate()
    
    F1, F3R, F3, F5R, F5, FPP = filters
    
    layer1x1_0 = Conv2D(F1, 1, activation='relu')(A_prev) #
    
    layer1x1_1 = Conv2D(F3R, 1, padding='same', activation='relu')(A_prev)
    
    layer3x3 = Conv2D(F3, 3, padding='same', activation='relu')(layer1x1_1) #
    
    layer1x1_2 = Conv2D(F5R, 1, padding='same', activation='relu')(A_prev)
    
    layer5x5 = Conv2D(F5, 5, padding='same',  activation='relu')(layer1x1_2) #
    
    layerMax = MaxPooling2D(1)(A_prev)
    
    layer1x1_3 = Conv2D(FPP, 1, padding='same', activation='relu')(layerMax) #
    
    layer_out = Concatenate([layer1x1_0, layer3x3, layer5x5, layer1x1_3])
    return layer_out
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 96) 384         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 224, 224, 16) 64          input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Co

In [ ]:
 [(None, 224, 224, 64), (None, 222, 222, 128), (None, 220, 220, 32), (None, 224, 224, 32)]

In [ ]:
# one by one from the table
def inception_network():
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dropout = K.layers.Dropout(rate=0.6)
    Dense = K.layers.Dense
    X = K.Input(shape=(224, 224, 3))
    
    layer7x7 = Conv2D(64, 7, 2, padding='same',  activation='relu')(X)
    layerMax = MaxPooling2D(3, 2, padding='same')(layer7x7)
    
    layer3x3 = Conv2D(64, 1, 1,  activation='relu')(layerMax)
    layer3x3 = Conv2D(192, 3, 1, padding='same',  activation='relu')(layer3x3)
    layerMax_2 = MaxPooling2D(3, 2, padding='same')(layer3x3)

    inception = inception_block(layerMax_2, [64, 96, 128, 16, 32, 32])
    inception = inception_block(inception, [128, 128, 192, 32, 96, 64])
    
    layerMax_2 = MaxPooling2D(3, 2, padding='same')(inception)
    inception = inception_block(layerMax_2, [192, 96, 208, 16, 48, 64])
    inception = inception_block(inception, [160, 112, 224, 24, 64, 64])
    inception = inception_block(inception, [128, 128, 256, 24, 64, 64])
    inception = inception_block(inception, [112, 144, 288, 32, 64, 64])
    inception = inception_block(inception, [256, 160, 320, 32, 128, 128])
    layerMax_2 = MaxPooling2D(3, 2, padding='same')(inception)
    inception = inception_block(layerMax_2, [256, 160, 320, 32, 128, 128])
    inception = inception_block(inception, [384, 192, 384, 48, 128, 128])
    layerAVG_2 = AveragePooling2D(7, 1)(inception)
    dropped = Dropout(layerAVG_2)
    # dropped = Dense(1000, activation="relu")(dropped)
    Y = Dense(1000, activation="softmax")(dropped)

    model = K.Model(inputs=X, outputs=Y)
    return model

if __name__ == '__main__':
    model = inception_network()
    model.summary()

In [5]:
def identity_block(A_prev, filters):
    """ doc """
    Conv2D = K.layers.Conv2D
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Add = K.layers.Add
    F11, F3, F12 = filters
    layer1x1 = Conv2D(F11, 1, padding='same', kernel_initializer='he_normal')(A_prev)
    layer1x1 = BatchNorm()(layer1x1)
    layer1x1 = Activation('relu')(layer1x1)

    layer3x3 = Conv2D(F3, 3, padding='same', kernel_initializer='he_normal')(layer1x1)
    layer3x3 = BatchNorm()(layer3x3)
    layer3x3 = Activation('relu')(layer3x3)

    layer1x1 = Conv2D(F12, 1, padding='same', kernel_initializer='he_normal')(layer3x3)
    layer1x1 = BatchNorm()(layer1x1)

    layer_out = Add()([layer1x1, A_prev])
    layer_out = Activation('relu')(layer_out)

    return layer_out

if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 256))
    Y = identity_block(X, [64, 64, 256])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 256 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 224, 224, 64) 16448       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 224, 224, 64) 256         conv2d_6[0][0]                   
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 224, 224, 64) 0           batch_normalization_6[0][0]      
__________________________________________________________________________________________________
conv2d_7 (

In [33]:
projection_block = __import__('3-projection_block').projection_block

def projection_block(A_prev, filters, s=2):
    """ doc """
    Conv2D = K.layers.Conv2D
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Add = K.layers.Add
    F11, F3, F12 = filters

    layer1x1 = Conv2D(F11, 1, s, padding='same',
                      kernel_initializer='he_normal')(A_prev)
    layer1x1 = BatchNorm()(layer1x1)
    layer1x1 = Activation('relu')(layer1x1)

    layer3x3 = Conv2D(F3, 3, padding='same',
                      kernel_initializer='he_normal')(layer1x1)
    layer3x3 = BatchNorm()(layer3x3)
    layer3x3 = Activation('relu')(layer3x3)

    layer1x1 = Conv2D(F12, 1, padding='same',
                      kernel_initializer='he_normal')(layer3x3)
    layer1x1 = BatchNorm()(layer1x1)  #
    
    layer1x1_s = Conv2D(F12, 1, s,
                      kernel_initializer='he_normal')(A_prev)
    layer1x1_s = BatchNorm()(layer1x1_s)  #

    layer_out = Add()([layer1x1, layer1x1_s])
    layer_out = Activation('relu')(layer_out)

    return layer_out
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = projection_block(X, [64, 64, 256])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1397 (Conv2D)            (None, 112, 112, 64) 256         input_29[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1397 (Batch (None, 112, 112, 64) 256         conv2d_1397[0][0]                
__________________________________________________________________________________________________
activation_1291 (Activation)    (None, 112, 112, 64) 0           batch_normalization_1397[0][0]   
__________________________________________________________________________________________________
conv2d_139

In [6]:
import tensorflow.keras as K
identity_block = __import__('2-identity_block').identity_block
projection_block = __import__('3-projection_block').projection_block

def resnet50():
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dense = K.layers.Dense
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation

    def layersConv(X, k, f, s=None, p='same'):
        layer = Conv2D(k, f, s, padding=p,
                       kernel_initializer='he_normal')(X)
        layer = BatchNorm()(layer)
        layer = Activation('relu')(layer)
        return layer

    X = K.Input(shape=(224, 224, 3))

    layer = layersConv(X, 64, 7, 2)

    layerMax = MaxPooling2D(3, 2, 'same')(layer)

    filters = [64, 64, 256]
    layer = projection_block(layer, filters, 1)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)

    filters = [128, 128, 512]
    layer = projection_block(layer, filters, 2)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)

    filters = [256, 256, 1024]
    layer = projection_block(layer, filters, 2)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)

    filters = [512, 512, 2048]
    layer = projection_block(layer, filters, 2)
    layer = identity_block(layer, filters)
    layer = identity_block(layer, filters)

    layerAVG = AveragePooling2D(7)(layer)

    Y = Dense(1000, kernel_initializer='he_normal', activation="softmax")(layerAVG)

    model = K.Model(inputs=X, outputs=Y)
    return model

def resnet50bbb():
    """Return a resnet50 model"""
    indata = K.layers.Input((224, 224, 3))
    out = K.layers.Conv2D(64, 7, 2, padding='same',
                          kernel_initializer='he_normal')(indata)
    out = K.layers.BatchNormalization()(out)
    out = K.layers.Activation('relu')(out)
    out = K.layers.MaxPool2D(3, 2, padding='same')(out)
    filters = [64, 64, 256]
    out = projection_block(out, filters, 1)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    filters = [128, 128, 512]
    out = projection_block(out, filters, 2)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    filters = [256, 256, 1024]
    out = projection_block(out, filters, 2)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    filters = [512, 512, 2048]
    out = projection_block(out, filters, 2)
    out = identity_block(out, filters)
    out = identity_block(out, filters)
    out = K.layers.AvgPool2D(7)(out)
    out = K.layers.Dense(1000, kernel_initializer='he_normal',
                         activation='softmax')(out)
    return K.Model(indata, out)

if __name__ == '__main__':
    model = resnet50()
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_159 (Conv2D)             (None, 112, 112, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_159 (BatchN (None, 112, 112, 64) 256         conv2d_159[0][0]                 
__________________________________________________________________________________________________
activation_147 (Activation)     (None, 112, 112, 64) 0           batch_normalization_159[0][0]    
__________________________________________________________________________________________________
conv2d_160

In [47]:
def dense_block(X, nb_filters, growth_rate, layers):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dense = K.layers.Dense
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Concatenate = K.layers.Concatenate
    def layersConv(X, k, f, p='valid', s=1):
        layer = Conv2D(k, f, s, padding=p,
                       kernel_initializer='he_normal')(X)
        return layer

    layer_prev = X
    for layer in range(layers):
        layer_new = BatchNorm()(layer_prev)
        layer_new = Activation('relu')(layer_new)
        # 4k X
        layer_new = layersConv(layer_new, growth_rate, 1, 'same')

        layer_new = BatchNorm()(layer_new)
        layer_new = Activation('relu')(layer_new)
        layer_new = layersConv(layer_new, growth_rate, 3, 'same')
        
        layer_prev = Concatenate()([layer_prev, layer_new])
        

        nb_filters += growth_rate

    return layer_prev, nb_filters

if __name__ == '__main__':
    X = K.Input(shape=(56, 56, 64))
    Y, nb_filters = dense_block(X, 64, 32, 6)
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()
    print(nb_filters)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, 56, 56, 64)   0                                            
__________________________________________________________________________________________________
batch_normalization_462 (BatchN (None, 56, 56, 64)   256         input_44[0][0]                   
__________________________________________________________________________________________________
activation_457 (Activation)     (None, 56, 56, 64)   0           batch_normalization_462[0][0]    
__________________________________________________________________________________________________
conv2d_465 (Conv2D)             (None, 56, 56, 32)   2080        activation_457[0][0]             
__________________________________________________________________________________________________
batch_norm

In [5]:
def transition_layer(X, nb_filters, compression):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dense = K.layers.Dense
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Concatenate = K.layers.Concatenate

    def layersConv(X, k, f, p='valid', s=1):
        layer = Conv2D(k, f, s, padding=p,
                       kernel_initializer='he_normal')(X)
        return layer

    layer_new = BatchNorm()(X)
    layer_new = Activation('relu')(layer_new)
    nb_filters = int(nb_filters * compression)
    layer_new = layersConv(layer_new, nb_filters, 1)
    layer_new = AveragePooling2D(2)(layer_new)

    return layer_new, nb_filters

if __name__ == '__main__':
    X = K.Input(shape=(56, 56, 256))
    Y, nb_filters = transition_layer(X, 256, 0.5)
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()
    print(nb_filters)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 56, 56, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 256)       1024      
_________________________________________________________________
activation_2 (Activation)    (None, 56, 56, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       32896     
_________________________________________________________________
average_pooling2d_1 (Average (None, 28, 28, 128)       0         
Total params: 33,920
Trainable params: 33,408
Non-trainable params: 512
_________________________________________________________________
128


In [7]:
dense_block = __import__('5-dense_block').dense_block
transition_layer = __import__('6-transition_layer').transition_layer


def densenet121(growth_rate=32, compression=1.0):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dense = K.layers.Dense
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation

    def layersConv(X, k, f, s=None, p='same'):
        layer = BatchNorm()(X)
        layer = Activation('relu')(layer)
        layer = Conv2D(k, f, s, padding=p,
                       kernel_initializer='he_normal')(layer)
        return layer

    X = K.Input(shape=(224, 224, 3))
    nb_filters = 64

    # Convolution
    layer = layersConv(X, nb_filters, 7, 2)

    # Pooling
    layerMax = MaxPooling2D(3, 2, padding='same')(layer)

    # Dense Block / Transition Layer (1)
    layers = 6
    layer, nb_filters = dense_block(layerMax, nb_filters, growth_rate, layers)
    layer, nb_filters = transition_layer(layer, nb_filters, compression)

    # Dense Block / Transition Layer (2)
    layers = 12
    layer, nb_filters = dense_block(layer, nb_filters, growth_rate, layers)
    layer, nb_filters = transition_layer(layer, nb_filters, compression)

    # Dense Block / Transition Layer (3)
    layers = 24
    layer, nb_filters = dense_block(layer, nb_filters, growth_rate, layers)
    layer, nb_filters = transition_layer(layer, nb_filters, compression)

    # Dense Block (3)
    layers = 16
    layer, nb_filters = dense_block(layer, nb_filters, growth_rate, layers)

    # Classification Layer (Pooling + 1000D fully-connected, softmax)
    layerAVG = AveragePooling2D(7)(layer)
    Y = Dense(1000, activation="softmax")(layerAVG)

    model = K.Model(inputs=X, outputs=Y)
    return model

if __name__ == '__main__':
    model = densenet121(32, 0.5)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
batch_normalization_480 (BatchN (None, 224, 224, 3)  12          input_5[0][0]                    
__________________________________________________________________________________________________
activation_480 (Activation)     (None, 224, 224, 3)  0           batch_normalization_480[0][0]    
__________________________________________________________________________________________________
conv2d_480 (Conv2D)             (None, 112, 112, 64) 9472        activation_480[0][0]             
__________________________________________________________________________________________________
max_poolin

In [32]:
#dense_block = __import__('5-dense_block').dense_block
def dense_block(X, nb_filters, growth_rate, layers):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    AveragePooling2D = K.layers.AveragePooling2D
    Conv2D = K.layers.Conv2D
    Dense = K.layers.Dense
    BatchNorm = K.layers.BatchNormalization
    Activation = K.layers.Activation
    Concatenate = K.layers.Concatenate

    def layersConv(X, k, f, p='valid', s=1):
        layer = BatchNorm()(X)
        layer = Activation('relu')(layer)
        layer = Conv2D(k, f, s, padding=p,
                       kernel_initializer='he_normal')(layer)
        return layer

    layer_prev = X
    for layer in range(layers):
        # 4k X
        layer_new = layersConv(layer_prev, growth_rate * 4, 1, 'same')
        layer_new = layersConv(layer_new, growth_rate, 3, 'same')
        layer_prev = Concatenate()([layer_prev, layer_new])
        nb_filters += growth_rate
    return layer_prev, nb_filters

print(nb_filters1)
X2 = K.Input(shape=(28, 28, 128))
Y2, nb_filters2 = dense_block(X2, 128, 32, 12)
model2 = K.models.Model(inputs=X2, outputs=Y2)
model2.summary()
print(nb_filters2)

256
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           (None, 28, 28, 128)  0                                            
__________________________________________________________________________________________________
batch_normalization_1218 (Batch (None, 28, 28, 128)  512         input_45[0][0]                   
__________________________________________________________________________________________________
activation_1218 (Activation)    (None, 28, 28, 128)  0           batch_normalization_1218[0][0]   
__________________________________________________________________________________________________
conv2d_1218 (Conv2D)            (None, 28, 28, 128)  16512       activation_1218[0][0]            
__________________________________________________________________________________________________
batch_